# Plugins

Plugins enhance the capabilities of language models (LLM) through modular execution, allowing you to:

1. Execute Deterministic Logic:
Each plugin contains specific and well-defined logic to perform concrete tasks.

2. Extend Functionality:
You can expand the capabilities of language models by integrating them with different tools and services.

3. Tailored Customization:
Plugins facilitate adapting functionality according to the needs of each user or application, offering flexibility and control when designing solutions.

Plugins allow you to enrich the LLM with additional functionalities in an orderly and modular manner, making the integration and extension of services simple and effective.

In [24]:
#r "nuget: Microsoft.SemanticKernel, 1.39.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.39.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.39.0"

Installed Packages Microsoft.SemanticKernel, 1.39.0 microsoft.semantickernel.connectors.openai, 1.39.0 microsoft.semantickernel.core, 1.39.0

In [25]:
using System;
using System.ComponentModel;
using System.IO;
using System.Threading;
using System.Threading.Tasks;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Agents;

using Kernel = Microsoft.SemanticKernel.Kernel;
using KernelPlugin = Microsoft.SemanticKernel.KernelPlugin;
using KernelExtensions = Microsoft.SemanticKernel.KernelExtensions;
using KernelPluginExtensions = Microsoft.SemanticKernel.KernelPluginExtensions;
using ChatCompletionAgent = Microsoft.SemanticKernel.Agents.AzureAI.ChatCompletionAgent;

Error: (19,61): error CS0234: The type or namespace name 'AzureAI' does not exist in the namespace 'Microsoft.SemanticKernel.Agents' (are you missing an assembly reference?)

In [26]:
using System.IO;
using System.Text.Json;

public class Settings
{
    public string Model { get; set; }
    public string AzureEndpoint { get; set; }
    public string ApiKey { get; set; }

    public static (string, string, string) LoadFromFile()
    {
        var jsonString = File.ReadAllText("config/settings.json");
        var options = new JsonSerializerOptions { PropertyNameCaseInsensitive = true };
        var settings = JsonSerializer.Deserialize<Settings>(jsonString, options);
        if (settings == null)
        {
            throw new Exception("Failed to load settings from the configuration file.");
        }
        return (settings.Model, settings.AzureEndpoint, settings.ApiKey);
    }
}

In [27]:
public class GreetingSkill
{
    [KernelFunction, Description("Greet the user based on its name")]
    public string Greetings(string name)
    {
        return $"Hello, {name}! It's nice to meet you!";
    }

    [KernelFunction, Description("Greet the user politely based on its name")]
    public string PoliteGreetings(string name)
    {
        return $"Hello, dear {name}! It's very pleasant to speak with you!";
    }

    [KernelFunction, Description("Greet the user using slang")]
    public string SlangGreetings(string name)
    {
        return $"Yo {name}! Who'd ya doing, daug?";
    }
}

In [ ]:
public class MathSkill
{
    [KernelFunction, Description("Sum Two Numbers")]
    public float Sum(float numberOne, float numberTwo)
    {
        return numberOne + numberTwo;
    }

    [KernelFunction, Description("Multiply Two Numbers")]
    public float Multiply(float numberOne, float numberTwo)
    {
        return numberOne * numberTwo;
    }

    [KernelFunction, Description("Calculates the Exponential of numberOne to the Power of numberTwo")]
    public float Exponential(float numberOne, float numberTwo)
    {
        return (float)Math.Pow(numberOne, numberTwo);
    }
}

In [29]:
public class ChatSkill
{
    [KernelFunction, Description("Chat with the user")]
    public string Chat(string input)
    {
        return $"You said: {input}";
    }
}

In [ ]:
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();
IKernelBuilder builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

builder.Plugins.AddFromType<GreetingSkill>("GreetingSkill");
builder.Plugins.AddFromType<MathSkill>("MathSkill");
builder.Plugins.AddFromType<ChatSkill>("ChatSkill");

OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Kernel kernel = builder.Build();
OpenAIPromptExecutionSettings executionSettings = openAIPromptExecutionSettings;
executionSettings.FunctionChoiceBehavior = FunctionChoiceBehavior.Auto();

var chatCompletion = kernel.GetRequiredService<IChatCompletionService>();
var chatHistory = new ChatHistory();

var chatCompletionResult = await chatCompletion.GetChatMessageContentAsync("How much is 512^12", executionSettings, kernel);

Console.WriteLine($"Result: {chatCompletionResult}\n");
chatHistory.AddAssistantMessage(chatCompletionResult.ToString());
Console.WriteLine($"Chat history: {JsonSerializer.Serialize(chatHistory)}\n");

Result: Hi there! I'm here and ready to assist you. How can I help you today?

Chat history: [{"Role":{"Label":"assistant"},"Items":[{"$type":"TextContent","Text":"Hi there! I\u0027m here and ready to assist you. How can I help you today?"}]}]

